In [1]:
import h5py as h5
import hdf5storage
import numpy as np
import imageio
import mat73
import os
import cv2
import random
import time
import argparse
#from IPython.display import Image

In [2]:
#Read Picture and return it

def readThePicture(picturepath):
    #  open ImageObject
    img = cv2.imread(picturepath, cv2.IMREAD_UNCHANGED)# | cv2.IMREAD_ANYCOLOR | cv2.IMREAD_ANYDEPTH)
    #old
    #imageio.plugins.freeimage.download()
    #img=imageio.imread(picturepath) #liest Bild von picturepath
    return(img) #returns picture

In [3]:
def tMO(file,name): #tonemapping the file
    try:
        if (name == 'reinhard'):
            tom = cv2.createTonemapReinhard()
        if (name == 'mantiuk'):
            tom = cv2.createTonemapMantiuk()
        if (name == 'drago'):
            tom = cv2.createTonemapDrago()
       # if (name == 'linear'):
        #    tom = cv2.createTonemap()
    except: 
        print('ToneMapping Error')
    ldr = tom.process(file)
    return ldr

In [4]:
def convert(img, target_type_min, target_type_max): # converts the input array to a target type with the bounderys given
    imin = img.min()
    imax = img.max()
    #print(imin)
    a = (target_type_max - target_type_min) / (imax - imin) # generates a factor a to multiply with the img
    b = target_type_max - a * imax
    new_img = (a * img + b)
    return new_img

In [5]:
def savePic(picture,fileName,extention,outPath): #saves the given array as a pictures to the given output path
    outPath = outPath+fileName+'.'+extention
    try:
        #old
        #imageio.imwrite(outPath,picture,format=extention)#extention'')#save the Data (path with name and file extention,PictureMatrix,format)
        #new
        cv2.imwrite(outPath,picture)
        print(outPath+'<=== Writing')
    except:
        print('Failed while saving picture: '+fileName+' to '+ outPath+' sorry :(')
        print('--------------------')

In [6]:
def cutPatch(begX,begY,endX,endY,picyx):#cuts out a array of a given array
    try: 
        picyx = picyx[begY:endY,begX:endX] #format y,start:End | x,start:End
    except:
        print('FormatMaking Failed')
    return picyx #returns a small part of the pic file

In [7]:
def Randtone_map():
        #a random tonemapping is returned
        rand = random.SystemRandom()
        tmNumber = round((rand.randint(0, 20)/10)) # generates a random tonempaiing nuber 
        try:
            if (tmNumber<=0):
                return 'reinhard' #retruns the name of the tonemapper
            if (tmNumber==1):
                return 'mantiuk'
            if (tmNumber>=2):
                return 'drago'
           # if (tmNumber>=3):
            #    return 'linear'
        except:
            print('there was an tmo Error')
#ToDo Output in CSV to later analize

In [8]:
def totalpatchespossible(path,amountOfPictures,extention,px,py,tokonvPic): #calculates the amount of total possible patches of the path you picked 
    arraysize = 0 #zero the output
    amountOfPictures = amountOfPictures - 1
    tokonvPic= (amountOfPictures-(tokonvPic))+1 #generates the stop argument 
    while (amountOfPictures) >= tokonvPic:
        try:
            helping = os.listdir(path)[amountOfPictures].split('.')[1] #reading the File extention
            if ( helping == extention): #only counts files with the same extention
                he = (path+str(os.listdir(path)[amountOfPictures])) #reading path to picture
                print(he) #prints the name of the picture what is currently been read
                readPic = readThePicture(he)
                arraysize = arraysize + (int(readPic.shape[1]/px)*int(readPic.shape[0]/py))# calculate the whole size an cut away the rest even when 0.9
        except:
            print('fail count all patches')
        amountOfPictures = amountOfPictures - 1
    print('There will be >> '+str(arraysize)+' << total patches')
    return arraysize         

In [9]:
def patchesxy(inputpic,px,py): #calculates how often the array can be devided by px in x and py in y
    arraysize = []
    try:
        y = int(inputpic.shape[0]/py)# calculates the number of patches in the Y-axses cuts the picture i
        x = int(inputpic.shape[1]/px)
        arraysize = (x,y)
    except:
        print('fail calc x and y')
    return arraysize
        

In [10]:
def resizePic(inputpic,factor): #reszizing the inputpic picture keeping the information but scaling it down
    y = int((inputpic.shape[0])/factor) #multiply the Factor
    x = int((inputpic.shape[1])/factor) #multiply the Factor
    pic = cv2.resize(inputpic,(x,y)) 
    return pic

In [11]:
def RGBtoYUV(img): #changeing the img picture from RGB- to YUV-Color space
    pictureYUV = cv2.cvtColor(img, cv2.COLOR_RGB2YUV)
    return pictureYUV

In [12]:
def YUVtoRGB(img):#changeing the img picture from YUV- to RGB-Color space
    pictureRGB = cv2.cvtColor(img, cv2.COLOR_YUV2RGB)
    return pictureRGB

In [13]:
#TO Finish
def inputargs():#todo finish
    parser = argparse.ArgumentParser()
    parser.add_argument('--foo', help='foo help')
    args = parser.parse_args()
    

In [14]:
# this function should return an random array with
def randArray(allPatches, patchPerPic): 
    patchRandArray = np.zeros((patchPerPic)) #generates the array
    rand = random.SystemRandom() #starts the rand generator
    while(patchPerPic > 0):
        patchnNum = round(rand.randint(0, allPatches)) #
        if (not(patchnNum in patchRandArray)): #if number is not in array already 
            patchRandArray[patchPerPic-1] = patchnNum #write it in
            patchPerPic = patchPerPic -1
    return patchRandArray

In [15]:
def returnPosFromNumber(xMax,yMax, pos): #should return one coordinat in x and y cunts up from 0 to pos-1
    #allpos = xMax*yMax
    pos = pos-1
    x = 0
    y = pos // xMax #modulo
    x = pos % xMax
    return x+1,y+1

In [18]:
#---- input section
#TO DO add parser
        
start_time = time.time() #start the timeing of the Prgramm
path = ''
print('This skript uses a folder and converts the pictures in smaler random Patches in YUV or RGB color space, Use same size file to avoid errors')
#data extention of the input data
extention = input('What fileextention is supposed to be put into patches(e.g. hdr, png) [default: hdr: ]') or 'hdr'
print(extention)
#Where are the rawData at?
path = input('Where is the Path to the pictures (should be .hdr) [default: ./hdrInput/]') or './hdrInput/'
if not os.path.exists(path):
        os.mkdir(path)
print(path)
inhalte = (os.listdir(path)) #list of all files in path
amountOfPictures = 0

amountOfPictures = sum(1 for f in os.listdir(path) if f.endswith('.'+extention)) #summ all data ending with right extention
print('There are: '+str(amountOfPictures)+' '+extention+' Pictures in the folder')
#User can choose how many pictures should be cut in patches
tokonvPic = int(input('how many Pictures do you want to cut into patches? default 1: ') or '1')
print(str(tokonvPic)+' pictures will be cut into patches')
#scale factor for the low resolution is inputed
factor = int(input('Scale factor for Ldr LR [default:2]: ') or "2")
#asks for the px size of the high resolution pictures
xaxis = int(input('X Patch size HDR HR [default:420px]: ') or "420")
print(xaxis)
yaxis = int(input('Y Patch size HDR HR [default:420px]: ') or "420")
print(yaxis)
#user can choose if the pacht-pictures should be in YU-V or RGB
youWantYUV = input('Do you want to convert to yuv default: no ') or 'no'
#user can coose in wich folder the .mat file is stored
savein = input('Should patches be saved in .mat file type: (m) oder should ist be saved as pictures (p) or saved as mat and .hdr/.png type: (mp), [default: m] ') or 'm'      
print(savein)
unit_varSdr = (np.float32)
unit_varHdr = (np.float32)
testing = input('Is the dataset for testing purposes or do you want to split the output pictures in cromagan single png pictures? default: no') or 'no'
if (savein == 'p' or savein == 'mp'):
    hrImgOut = input('Should the hdr pictures have the format hdr(yes) or png 16bit(no)? default: no (png 16bit)') or 'no'
if (savein == 'm' or savein == 'mp'):
    #user can choose the name for the .mat file
    matName = input('Input Mat name default: data ') or 'data'
    matPath = input('Input Mat directory path: ./matOut/ ') or './matOut/'
    if not os.path.exists(matPath):
            os.mkdir(matPath)
    jsi = input('Is it for the JSI-GAN converion from float32 to uint8/16? default: no ') or 'no'
    if (jsi != 'no'):
        unit_varSdr = (np.uint8)
        print('SDR .mat file will be uint8')
        unit_varHdr = (np.uint16)
        print('HDR .mat file will be uint16')
if (savein == 'p' or savein == 'mp'): #if user wants to output pates in picters he can choose where
    outPathsdr = input('spezify the output path of sdr pictures [default: ./sdrOut/ ] ') or './sdrOut/' #set the picture save path if it is choosen
    if not os.path.exists(outPathsdr):
            os.mkdir(outPathsdr)
    outPathhdr = input('spezify the output path of sdr pictures [default: ./hdrOut/ ] ') or './hdrOut/' #set the picture save path if it is choosen
    if not os.path.exists(outPathhdr):
        os.mkdir(outPathhdr)

        #TO DO if files should all have the same name or original Filename
inhalte
if ((input('do you want to know all patches possible? default: no') or 'no')!='no'):
    allpatches = totalpatchespossible(path,amountOfPictures,extention,xaxis,yaxis,tokonvPic)  #calc all output patches
patchAmount = input('How many patches do you want to cut out of each Picture? default: 30- ') or '30'

This skript uses a folder and converts the pictures in smaler random Patches in YUV or RGB color space, Use same size file to avoid errors
What fileextention is supposed to be put into patches(e.g. hdr, png) [default: hdr: ]
hdr
Where is the Path to the pictures (should be .hdr) [default: ./hdrInput/]
./hdrInput/
There are: 367 hdr Pictures in the folder
how many Pictures do you want to cut into patches? default 1: 50
50 pictures will be cut into patches
Scale factor for Ldr LR [default:2]: 
X Patch size HDR HR [default:420px]: 
420
Y Patch size HDR HR [default:420px]: 
420
Do you want to convert to yuv default: no y
Should patches be saved in .mat file type: (m) oder should ist be saved as pictures (p) or saved as mat and .hdr/.png type: (mp), [default: m] mp
mp
Is the dataset for testing purposes or do you want to split the output pictures in cromagan single png pictures? default: no
Should the hdr pictures have the format hdr(yes) or png 16bit(no)? default: no (png 16bit)
Input Mat 

In [19]:
### write pic to .mat and/or .hdr/.png

#Just for general information Data Structure JSI-Gan
###['SDR_data'][79][79][2][39839] dtype=uint8 Strukture .mat Data
###['HDR_data'][79][79][2][39839] dtype=uint16 Strukture .mat Data
    

#---- programm section

allpatches = int(patchAmount)*int(tokonvPic) # calculates the amount of pictures total
print('That will be ==> 'str(allpatches)+' Patches in total :)')
hdrarray = np.zeros((xaxis,yaxis,3,allpatches))#create empty np array of the size of allpatches
hdrarray = hdrarray.astype(unit_varHdr) #changes the type of np array to uint16
xldr = int(xaxis/factor) #calculates the samler array axes x
yldr = int(yaxis/factor)#calculates the samler array axes y
sdrarray = np.zeros((xldr,yldr,3,allpatches)) # creates the np array for the LR SDR array with new axes
sdrarray = sdrarray.astype(unit_varSdr)#changes the type of np array to uint8
#Arrays are defined in [amountOfallPatchesPossible,x,y,RGB]

print('Start processing...')
tokonvPic= (int(amountOfPictures)-int(tokonvPic))# the amount of pictures cut into pachtes is calculated
#allpatchesSave = allpatches #all pachtes are saved
print(tokonvPic)
#amountOfPictures = amountOfPictures - 1 # needs to count down because while counts to more than in amonut of Pictures
print(amountOfPictures)
while (amountOfPictures > tokonvPic):#tokonvPic): #filling Array with pachtes from high to low, beginning with the hightes Number
    currentFile = os.listdir(path)[amountOfPictures] #currentFile holds the name of the current position file 
    try:
        if (currentFile.split('.')[1] == str(extention)): #checks if file is ending as wanted
            he = (path+str(currentFile))#gives the path and name with extention of the to process file
            print('processing the picture: '+he) #prints it to the user
            originalPicture = readThePicture(he) #reads the currentpicture and saves it
            
            #print(originalPicture.max())
            #begX = 1 #startingnumber in wich the cutting starts in X-axes
            #endX = xaxis # sets current start pixel position at wich the cutting starts in X xaxis is the user defined size
            
            pxy=patchesxy(originalPicture,xaxis,yaxis) # gives back the length of the current picture (numx,numy) e.g. (3,2)
            px= pxy[0] #saves the max x pos in px
            py= pxy[1] #saves the max y pos in py
            #print(pxy)
            patchCuts = randArray((px*py),int(patchAmount))# returns a array with amount patchAmount and the random positions to cut
            #print(patchCuts)
            
            aktPatch = 0
            savePXY = px,py
            while (aktPatch < int(patchAmount)): # cut until you are at the beginning of the picture X position
                #print(str(patchAmount)+'Patchamaount')
                #print(str(patchCuts[(aktPatch)]))
                randPosXY = returnPosFromNumber((savePXY[0]),(savePXY[1]),int(patchCuts[(aktPatch)])) #returns the x,y coordinate within a given position
                aktPatch = aktPatch + 1
                begy = randPosXY[1]* yaxis #is the new begin Pos in y
                begx = randPosXY[0]* xaxis #is the new begin Pos in x
                #print('Yschnitt'+str(begy))
                #print('Xschnitt'+str(begx))
                #print(originalPicture.shape)
                px = begx - xaxis #is the new end Pos in x
                py = begy - yaxis #is the new end Pos in Y
                #begY = 1 #startingnumber in wich the cutting starts in Y-axes
                #endY = yaxis # sets current start pixel position at wich the cutting starts in Y
                #x = (begX*xaxis)-xaxis #start patch position in x

                #while (py>=begY): # cut until you are at the beginning of the picture Y position
                    ### reading the picture and cutting section
                    #y = (begY*yaxis)-yaxis #start patch position in y
                patch = cutPatch(px,py,begx,begy,originalPicture) #make the patch and return it to the patch (floart64) array
                #print(patch.shape)  
                tmo = Randtone_map() # returns a random name for the tonemapping Operator to use
                    
                patchLR = resizePic(patch,factor) #resize picture from Patch with the factor
                tmoed = tMO(patchLR,tmo) #tonemap the resized patch with the before chosen Tone Mapping Operator
                    #print(tmoed)
                patch_lrtm = tmoed
                    ####Color YUV Section
                if (youWantYUV != 'no'): #if change to yuv
                    patch_lrtm = RGBtoYUV(tmoed) # changes the color space of lr sdr array
                    patch = RGBtoYUV(patch) # changes the color space of hr hdr array
                    ###Picture section
                    ### Pictures will be saved as 8bit .png with the picture beeing between 0- 255 uint8
                    ### and 16 bit .hdr with the picture beeing between 0- 2^16 float16
                    #print('HDR MAX')
                if (savein == 'p' or savein == 'mp'): #save as picture if chosen
                        #buildFilename = ((currentFile.split('.')[0])+'_'+str(allpatches-1))#builds output name 
                    buildFilename = str(allpatches-1).zfill(6)# gives the filename only an number filled up with 6 zeros (mybe better if zeros from max allpatches)
                        # TODO Add a Input for the wanted out_format
                    patch_lrtmp = patch_lrtm * ((2 ** 8)-1)
                    if(testing != 'no'):
                        spaceIndi = 'u','v','y' #orders the Name to the right place
                        savePic((patch_lrtmp[:,:,0]),(str(allpatches-1)+'-'+spaceIndi[0]),'png',outPathsdr)#saves final singel color channel Picture
                        savePic((patch_lrtmp[:,:,1]),(str(allpatches-1)+'-'+spaceIndi[1]),'png',outPathsdr)#saves final singel color channel Picture
                        savePic((patch_lrtmp[:,:,2]),(str(allpatches-1)+'-'+spaceIndi[2]),'png',outPathsdr)#saves final singel color channel Picture
                            
                            ####Saveing the 16Bit HDR picturespatches
                        if(hrImgOut !='no'):
                            savePic((patch[:,:,0]),(str(allpatches-1)+'-'+spaceIndi[0]),'hdr',outPathhdr)#saves final singel color channel Picture
                            savePic((patch[:,:,1]),(str(allpatches-1)+'-'+spaceIndi[1]),'hdr',outPathhdr)#saves final singel color channel Picture
                            savePic((patch[:,:,2]),(str(allpatches-1)+'-'+spaceIndi[2]),'hdr',outPathhdr)#saves final singel color channel Picture
                            
                            
                            #Saveing the 16Bit PNG output picturepachtes
                        if(hrImgOut == 'no'):
                            patchbic = patch * ((2 ** 16)-1)
                            patchbic = patchbic.astype(np.uint16)
                            print('(HDR)-PNG is 16 bit')
                            savePic((patchbic[:,:,0]),(str(allpatches-1)+'-'+spaceIndi[0]),'png',outPathhdr)#saves final singel color channel Picture
                            savePic((patchbic[:,:,1]),(str(allpatches-1)+'-'+spaceIndi[1]),'png',outPathhdr)#saves final singel color channel Picture
                            savePic((patchbic[:,:,2]),(str(allpatches-1)+'-'+spaceIndi[2]),'png',outPathhdr)#saves final singel color channel Picture
                            ###To DO make 16 bit PNG
                    if(testing == 'no'):
                        if(hrImgOut == 'no'):
                            patchpic = patch * ((2 ** 16)-1)
                            patchpic = patchpic.astype(np.uint16)
                            savePic(patchpic,buildFilename,'png',outPathhdr)#change 'hdr' here for different HDR-picture save
                        if(hrImgOut != 'no'):
                            savePic(patch,buildFilename,'hdr',outPathhdr)#change 'hdr' here for different HDR-picture save    
                        savePic(patch_lrtmp,buildFilename,'png',outPathsdr)#chnage 'png' here for different LDR-picture save 

                if(unit_varHdr == np.uint16):
                    patch_lrtm = patch_lrtm * ((2 ** 8)-1) 
                    patch = patch * ((2 ** 16)-1) #HDR is changed so it fitts the unit16 format and can be compared
                        #print('HDR')
                        #print(patch.max())
                        #print(patch.min())
                        #print('LDR')
                        #print(patch_lr.max())
                        #print(patch_lr.min())
                    patch_lr = patch_lrtm.astype(unit_varSdr) #changes the data type in according to JSI Gan spec to uint8 for lr sdr
                    patch = patch.astype(unit_varHdr) #changes the data type in according to JSI Gan spec to uint16 for hr hdr
                    
                    
                    ###writing complete array section 
                p = (allpatches-1) #calcualte current patch position
                if (savein == 'm' or savein == 'mp'):
                        #if(unit_varHdr == np.uint8):
                        #patch_lrtm = patch_lrtm * ((2 ** 8)-1)
                    try:
                            #print(patch_lrtm.shape)
                        sdrarray[:,:,:,p] = patch_lrtm # try Write the tmoed Patch to sdrarray at current patch position
                    except:
                        print('Error at SDR- Array Writing :..(')
                    try:
                        hdrarray[:,:,:,p] = patch # try Write the hr hdr Patch to hdrarray at current patch position
                    except:
                        print('Error at HDR- Array Writing :..(')
                        
                        
                    #begY = begY + 1 #Counts up from 1 to all possible cuts of the current picture in Y
                    #endY = endY + yaxis #Counts up the position from endY in yaxis steps
                allpatches = allpatches - 1 #Counts down all patches of all pictures to 0
                print('Patch === '+str(allpatches)+' ==> Done')
                #begX = begX + 1 #Counts up from 1 to all possible cuts of the current picture in X
                #endX = endX + xaxis # Counts up the position from endX in xaxis steps
                
    except:
        print('Error with data maybe not an .hdr file continuing...')
        print(str(originalPicture.shape)+'OrgPicShape')
        print('BeginHereX '+str(begx)+' to '+str(px))
        print('beginHereY '+str(begy)+' to '+str(py))
        print(str(patchCuts.shape)+' PatchCutsAmaount')
        print(str(aktPatch)+' PatchPos')
        print(str(patchCuts[(aktPatch)])+' PatchCuts_pos')
        print(str(patch.shape)+' hdrPatchShape'
        print(str(patch_lrtm.shape)+' sdrPatchShape'
        
    amountOfPictures = amountOfPictures - 1 #counts down current picture pos
    #print(str(amountOfPictures)+'AmountOf Pic')
    
    
if (savein == 'mp' or savein == 'm' ): #only makes a Matlap File if wanted
    try:
        matLabel = 'HDR_data'
        if(testing != 'no'):
            matLabel = 'HDR_YUV'
            print('testdata')
        # Write TO HDR.Mat File
        h5.get_config().default_file_mode = 'a' #write enable
        matfilehdrdataHDR = {} # make a dictionary to store the MAT data in
        print('HDR Matlab file will have the format')
        print(hdrarray.shape)
        matfilehdrdataHDR[u''+matLabel] = hdrarray #save hdr array in that dictonary
        print('Writing HDR_'+matName+'.mat File to: '+ matPath)
        hdf5storage.write(matfilehdrdataHDR, '.', matPath+'HDR_'+matName+'.mat', matlab_compatible=True) #output the .mat data file
        print('Saved the HDR .mat file')

        #####Writing SDR .mat 
        #Switches the first with the last array Field 
        matfilesdrdatasdr = {} # make a dictionary to store the MAT data in
        matLabel = 'SDR_data'
        if(testing != 'no'):
            matLabel = 'SDR_YUV'
            print('testdata')
        print('SDR Matlab file will have the format')
        print(sdrarray.shape)
        matfilesdrdatasdr[u''+matLabel] = sdrarray #save sdr array in that dictonary
        print('Writing SDR_'+matName+'.mat File to: '+ matPath)
        hdf5storage.write(matfilesdrdatasdr, '.', matPath+'SDR_'+matName+'.mat', matlab_compatible=True) #output the .mat data file
        print('Saved the SDR .mat file')
    except:
        print('error at writing matlab file sorry :(')

    sdrpro = (np.count_nonzero(sdrarray)/sdrarray.size)*100        
    print(str(sdrpro)+'% of SDRarray numbers are bigger than 0')

    sdrpro = (np.count_nonzero(hdrarray)/hdrarray.size)*100        
    print(str(sdrpro)+'% of HDRarray numbers are bigger than 0')

print(str((time.time() - start_time)/60)+' Minutes') #outputs the time in minutes
print('------------------------- Done --------------------')

1500all Patches
Start processing...
317
367
processing the picture: ./hdrInput/kiara_2_sunrise_16k.hdr
545.0
(420, 420, 3)
./hdrOut/001499.png<=== Writing
./sdrOut/001499.png<=== Writing
Patch === 1499 ==> Done
636.0
(420, 420, 3)
./hdrOut/001498.png<=== Writing
./sdrOut/001498.png<=== Writing
Patch === 1498 ==> Done
247.0
(420, 420, 3)
./hdrOut/001497.png<=== Writing
./sdrOut/001497.png<=== Writing
Patch === 1497 ==> Done
20.0
(420, 420, 3)
./hdrOut/001496.png<=== Writing
./sdrOut/001496.png<=== Writing
Patch === 1496 ==> Done
417.0
(420, 420, 3)
./hdrOut/001495.png<=== Writing
./sdrOut/001495.png<=== Writing
Patch === 1495 ==> Done
477.0
(420, 420, 3)
./hdrOut/001494.png<=== Writing
./sdrOut/001494.png<=== Writing
Patch === 1494 ==> Done
285.0
(420, 420, 3)
./hdrOut/001493.png<=== Writing
./sdrOut/001493.png<=== Writing
Patch === 1493 ==> Done
330.0
(420, 420, 3)
./hdrOut/001492.png<=== Writing
./sdrOut/001492.png<=== Writing
Patch === 1492 ==> Done
431.0
(420, 420, 3)
./hdrOut/00149

./hdrOut/001419.png<=== Writing
./sdrOut/001419.png<=== Writing
Patch === 1419 ==> Done
297.0
(420, 420, 3)
./hdrOut/001418.png<=== Writing
./sdrOut/001418.png<=== Writing
Patch === 1418 ==> Done
147.0
(420, 420, 3)
./hdrOut/001417.png<=== Writing
./sdrOut/001417.png<=== Writing
Patch === 1417 ==> Done
737.0
(420, 420, 3)
./hdrOut/001416.png<=== Writing
./sdrOut/001416.png<=== Writing
Patch === 1416 ==> Done
222.0
(420, 420, 3)
./hdrOut/001415.png<=== Writing
./sdrOut/001415.png<=== Writing
Patch === 1415 ==> Done
237.0
(420, 420, 3)
./hdrOut/001414.png<=== Writing
./sdrOut/001414.png<=== Writing
Patch === 1414 ==> Done
384.0
(420, 420, 3)
./hdrOut/001413.png<=== Writing
./sdrOut/001413.png<=== Writing
Patch === 1413 ==> Done
15.0
(420, 420, 3)
./hdrOut/001412.png<=== Writing
./sdrOut/001412.png<=== Writing
Patch === 1412 ==> Done
343.0
(420, 420, 3)
./hdrOut/001411.png<=== Writing
./sdrOut/001411.png<=== Writing
Patch === 1411 ==> Done
39.0
(420, 420, 3)
./hdrOut/001410.png<=== Writin

Patch === 1344 ==> Done
669.0
(420, 420, 3)
./hdrOut/001343.png<=== Writing
./sdrOut/001343.png<=== Writing
Patch === 1343 ==> Done
422.0
(420, 420, 3)
./hdrOut/001342.png<=== Writing
./sdrOut/001342.png<=== Writing
Patch === 1342 ==> Done
339.0
(420, 420, 3)
./hdrOut/001341.png<=== Writing
./sdrOut/001341.png<=== Writing
Patch === 1341 ==> Done
402.0
(420, 420, 3)
./hdrOut/001340.png<=== Writing
./sdrOut/001340.png<=== Writing
Patch === 1340 ==> Done
262.0
(420, 420, 3)
./hdrOut/001339.png<=== Writing
./sdrOut/001339.png<=== Writing
Patch === 1339 ==> Done
646.0
(420, 420, 3)
./hdrOut/001338.png<=== Writing
./sdrOut/001338.png<=== Writing
Patch === 1338 ==> Done
134.0
(420, 420, 3)
./hdrOut/001337.png<=== Writing
./sdrOut/001337.png<=== Writing
Patch === 1337 ==> Done
520.0
(420, 420, 3)
./hdrOut/001336.png<=== Writing
./sdrOut/001336.png<=== Writing
Patch === 1336 ==> Done
75.0
(420, 420, 3)
./hdrOut/001335.png<=== Writing
./sdrOut/001335.png<=== Writing
Patch === 1335 ==> Done
270.0

./hdrOut/001267.png<=== Writing
./sdrOut/001267.png<=== Writing
Patch === 1267 ==> Done
705.0
(420, 420, 3)
./hdrOut/001266.png<=== Writing
./sdrOut/001266.png<=== Writing
Patch === 1266 ==> Done
546.0
(420, 420, 3)
./hdrOut/001265.png<=== Writing
./sdrOut/001265.png<=== Writing
Patch === 1265 ==> Done
297.0
(420, 420, 3)
./hdrOut/001264.png<=== Writing
./sdrOut/001264.png<=== Writing
Patch === 1264 ==> Done
440.0
(420, 420, 3)
./hdrOut/001263.png<=== Writing
./sdrOut/001263.png<=== Writing
Patch === 1263 ==> Done
264.0
(420, 420, 3)
./hdrOut/001262.png<=== Writing
./sdrOut/001262.png<=== Writing
Patch === 1262 ==> Done
223.0
(420, 420, 3)
./hdrOut/001261.png<=== Writing
./sdrOut/001261.png<=== Writing
Patch === 1261 ==> Done
329.0
(420, 420, 3)
./hdrOut/001260.png<=== Writing
./sdrOut/001260.png<=== Writing
Patch === 1260 ==> Done
processing the picture: ./hdrInput/kloppenheim_07_16k.hdr
474.0
(420, 420, 3)
./hdrOut/001259.png<=== Writing
./sdrOut/001259.png<=== Writing
Patch === 1259

Patch === 1193 ==> Done
388.0
(420, 420, 3)
./hdrOut/001192.png<=== Writing
./sdrOut/001192.png<=== Writing
Patch === 1192 ==> Done
301.0
(420, 420, 3)
./hdrOut/001191.png<=== Writing
./sdrOut/001191.png<=== Writing
Patch === 1191 ==> Done
102.0
(420, 420, 3)
./hdrOut/001190.png<=== Writing
./sdrOut/001190.png<=== Writing
Patch === 1190 ==> Done
68.0
(420, 420, 3)
./hdrOut/001189.png<=== Writing
./sdrOut/001189.png<=== Writing
Patch === 1189 ==> Done
237.0
(420, 420, 3)
./hdrOut/001188.png<=== Writing
./sdrOut/001188.png<=== Writing
Patch === 1188 ==> Done
714.0
(420, 420, 3)
./hdrOut/001187.png<=== Writing
./sdrOut/001187.png<=== Writing
Patch === 1187 ==> Done
243.0
(420, 420, 3)
./hdrOut/001186.png<=== Writing
./sdrOut/001186.png<=== Writing
Patch === 1186 ==> Done
669.0
(420, 420, 3)
./hdrOut/001185.png<=== Writing
./sdrOut/001185.png<=== Writing
Patch === 1185 ==> Done
51.0
(420, 420, 3)
./hdrOut/001184.png<=== Writing
./sdrOut/001184.png<=== Writing
Patch === 1184 ==> Done
66.0
(

...

./sdrOut/000323.png<=== Writing
Patch === 323 ==> Done
444.0
(420, 420, 3)
./hdrOut/000322.png<=== Writing
./sdrOut/000322.png<=== Writing
Patch === 322 ==> Done
456.0
(420, 420, 3)
./hdrOut/000321.png<=== Writing
./sdrOut/000321.png<=== Writing
Patch === 321 ==> Done
268.0
(420, 420, 3)
./hdrOut/000320.png<=== Writing
./sdrOut/000320.png<=== Writing
Patch === 320 ==> Done
622.0
(420, 420, 3)
./hdrOut/000319.png<=== Writing
./sdrOut/000319.png<=== Writing
Patch === 319 ==> Done
624.0
(420, 420, 3)
./hdrOut/000318.png<=== Writing
./sdrOut/000318.png<=== Writing
Patch === 318 ==> Done
494.0
(420, 420, 3)
./hdrOut/000317.png<=== Writing
./sdrOut/000317.png<=== Writing
Patch === 317 ==> Done
348.0
(420, 420, 3)
./hdrOut/000316.png<=== Writing
./sdrOut/000316.png<=== Writing
Patch === 316 ==> Done
388.0
(420, 420, 3)
./hdrOut/000315.png<=== Writing
./sdrOut/000315.png<=== Writing
Patch === 315 ==> Done
364.0
(420, 420, 3)
./hdrOut/000314.png<=== Writing
./sdrOut/000314.png<=== Writing
Patch

./hdrOut/000242.png<=== Writing
./sdrOut/000242.png<=== Writing
Patch === 242 ==> Done
131.0
(420, 420, 3)
./hdrOut/000241.png<=== Writing
./sdrOut/000241.png<=== Writing
Patch === 241 ==> Done
695.0
(420, 420, 3)
./hdrOut/000240.png<=== Writing
./sdrOut/000240.png<=== Writing
Patch === 240 ==> Done
processing the picture: ./hdrInput/old_bus_depot_16k.hdr
228.0
(420, 420, 3)
./hdrOut/000239.png<=== Writing
./sdrOut/000239.png<=== Writing
Patch === 239 ==> Done
143.0
(420, 420, 3)
./hdrOut/000238.png<=== Writing
./sdrOut/000238.png<=== Writing
Patch === 238 ==> Done
60.0
(420, 420, 3)
./hdrOut/000237.png<=== Writing
./sdrOut/000237.png<=== Writing
Patch === 237 ==> Done
724.0
(420, 420, 3)
./hdrOut/000236.png<=== Writing
./sdrOut/000236.png<=== Writing
Patch === 236 ==> Done
517.0
(420, 420, 3)
./hdrOut/000235.png<=== Writing
./sdrOut/000235.png<=== Writing
Patch === 235 ==> Done
603.0
(420, 420, 3)
./hdrOut/000234.png<=== Writing
./sdrOut/000234.png<=== Writing
Patch === 234 ==> Done
5

./hdrOut/000165.png<=== Writing
./sdrOut/000165.png<=== Writing
Patch === 165 ==> Done
361.0
(420, 420, 3)
./hdrOut/000164.png<=== Writing
./sdrOut/000164.png<=== Writing
Patch === 164 ==> Done
445.0
(420, 420, 3)
./hdrOut/000163.png<=== Writing
./sdrOut/000163.png<=== Writing
Patch === 163 ==> Done
456.0
(420, 420, 3)
./hdrOut/000162.png<=== Writing
./sdrOut/000162.png<=== Writing
Patch === 162 ==> Done
191.0
(420, 420, 3)
./hdrOut/000161.png<=== Writing
./sdrOut/000161.png<=== Writing
Patch === 161 ==> Done
352.0
(420, 420, 3)
./hdrOut/000160.png<=== Writing
./sdrOut/000160.png<=== Writing
Patch === 160 ==> Done
99.0
(420, 420, 3)
./hdrOut/000159.png<=== Writing
./sdrOut/000159.png<=== Writing
Patch === 159 ==> Done
134.0
(420, 420, 3)
./hdrOut/000158.png<=== Writing
./sdrOut/000158.png<=== Writing
Patch === 158 ==> Done
278.0
(420, 420, 3)
./hdrOut/000157.png<=== Writing
./sdrOut/000157.png<=== Writing
Patch === 157 ==> Done
735.0
(420, 420, 3)
./hdrOut/000156.png<=== Writing
./sdrO

328.0
(420, 420, 3)
./hdrOut/000089.png<=== Writing
./sdrOut/000089.png<=== Writing
Patch === 89 ==> Done
389.0
(420, 420, 3)
./hdrOut/000088.png<=== Writing
./sdrOut/000088.png<=== Writing
Patch === 88 ==> Done
461.0
(420, 420, 3)
./hdrOut/000087.png<=== Writing
./sdrOut/000087.png<=== Writing
Patch === 87 ==> Done
522.0
(420, 420, 3)
./hdrOut/000086.png<=== Writing
./sdrOut/000086.png<=== Writing
Patch === 86 ==> Done
450.0
(420, 420, 3)
./hdrOut/000085.png<=== Writing
./sdrOut/000085.png<=== Writing
Patch === 85 ==> Done
662.0
(420, 420, 3)
./hdrOut/000084.png<=== Writing
./sdrOut/000084.png<=== Writing
Patch === 84 ==> Done
588.0
(420, 420, 3)
./hdrOut/000083.png<=== Writing
./sdrOut/000083.png<=== Writing
Patch === 83 ==> Done
26.0
(420, 420, 3)
./hdrOut/000082.png<=== Writing
./sdrOut/000082.png<=== Writing
Patch === 82 ==> Done
492.0
(420, 420, 3)
./hdrOut/000081.png<=== Writing
./sdrOut/000081.png<=== Writing
Patch === 81 ==> Done
76.0
(420, 420, 3)
./hdrOut/000080.png<=== Writ

Patch === 8 ==> Done
740.0
(420, 420, 3)
./hdrOut/000007.png<=== Writing
./sdrOut/000007.png<=== Writing
Patch === 7 ==> Done
86.0
(420, 420, 3)
./hdrOut/000006.png<=== Writing
./sdrOut/000006.png<=== Writing
Patch === 6 ==> Done
288.0
(420, 420, 3)
./hdrOut/000005.png<=== Writing
./sdrOut/000005.png<=== Writing
Patch === 5 ==> Done
93.0
(420, 420, 3)
./hdrOut/000004.png<=== Writing
./sdrOut/000004.png<=== Writing
Patch === 4 ==> Done
731.0
(420, 420, 3)
./hdrOut/000003.png<=== Writing
./sdrOut/000003.png<=== Writing
Patch === 3 ==> Done
363.0
(420, 420, 3)
./hdrOut/000002.png<=== Writing
./sdrOut/000002.png<=== Writing
Patch === 2 ==> Done
633.0
(420, 420, 3)
./hdrOut/000001.png<=== Writing
./sdrOut/000001.png<=== Writing
Patch === 1 ==> Done
706.0
(420, 420, 3)
./hdrOut/000000.png<=== Writing
./sdrOut/000000.png<=== Writing
Patch === 0 ==> Done
HDR Matlab file will have the format
(420, 420, 3, 1500)
Writing HDR_data.mat File to: ./matOut/
Saved the HDR .mat file
SDR Matlab file will

In [16]:
#python3 main.py --phase train --scale_factor 2 --train_data_path_LR_SDR ./SDR_data.mat --train_data_path_HR_HDR ./HDR_data.mat --epoch 5 --batch_size 4

In [21]:
### gives the amount of nuberst larger than 0 in percent
#np.count_nonzero(ab)
(np.count_nonzero(sdrarray)/sdrarray.size)*100

0.0

In [20]:
#np.count_nonzero(ab)
(np.count_nonzero(hdrarray)/hdrarray.size)*100

99.99995766787747

In [21]:
#python3 train.py --batch_size 100 -d ./hdrData -s ./pathToSaceChepoints 

In [20]:
#python3 main.py --phase test_mat --scale_factor 2 --train_data_path_LR_SDR ./SDR_data.mat --train_data_path_HR_HDR ./HDR_data.mat --test_data_path_LR_SDR /home/nico/programm/MdataPicPrep/matOut/SDR_data.mat --test_data_path_HR_HDR /home/nico/programm/MdataPicPrep/matOut/HDR_data.mat --batch_size 4